In [13]:
#import the following modules
import numpy as np
import Tkinter, Tkconstants, tkFileDialog
import warnings
import pandas as pd

from easygui import *
from imblearn.over_sampling import SMOTE
from Tkinter import *
from sklearn import preprocessing
from sklearn.preprocessing import Imputer
le = preprocessing.LabelEncoder()

#Use Tkinter to choose the file directory and save the local address as filepath
Tk().withdraw()
filepath = tkFileDialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("csv files","xlsx files"),("all files","*.*")))

C:/Users/310249682/Documents/data.xlsx


In [31]:
#Class to do propressing of the original data
class propressing:
    def __init__(self,data):
        self.data = data
        #delete useless features/variables from the data
        self.deductions( self.data )
        #transfer the * sign to NaN, in order to do the transform correctly
        self.data['Age'] = self.data['Age'].replace('*',np.nan)
        #data transform process.
        #If the feature is unicode, then transformed to 0,1,2,.... and replace the NaN with a new category.
        #If the feature is float/int, then replace the NaN as the mean value.
        for i in self.data.columns:
            for k in self.data[i][pd.notnull(self.data[i])]:
                ddtype = type(k)
                break
            if ddtype is unicode:
                le.fit( self.data[i] )
                self.data[i] = le.transform( self.data[i] )
            else:
                imp = Imputer(missing_values='NaN', strategy='mean',axis = 1)
                imp.fit(self.data[i])
                self.data[i] = imp.transform( self.data[i] )[0]
        #The outcome has four values:NaN-0,Alive-1,Dead-2,Unknow-3.We only select Alive and Dead Group.
        out_alive = self.data[self.data['OUTCOME'] == 1]
        out_dead = self.data[self.data['OUTCOME'] == 2]
        data_new = pd.concat([out_alive,out_dead])
        #Predictors name with deleting the OUTCOME feature.
        predictors = self.data.columns
        predictors = list(predictors)
        del predictors[32] 
        #Final data
        self.datapd = data_new
    
    def deductions( self,data ):
        #Delete the Time/Date-related features.
        del self.data['Day of Week']
        del self.data['Procedure Year']
        del self.data['Procedure Quarter of year']
        del self.data['Procedure Month']
        del self.data['Procedure MonthNumber']
        del self.data['Procedure Week of year']
        del self.data['Procedure Day of Month']
        #Delete the features with large number of NaN values. Mostly missing over 20000 of 30000.
        del self.data['HISTORY_HYPERTENSION']
        del self.data['TOTAL_CHOLESTEROL']
        del self.data['CATH EVENTS']
        del self.data['EJECTION_FRACTION']
        del self.data['WEIGHT']
        del self.data['HEIGHT']
        del self.data['HEART_RATE']
        del self.data['Euroscore']
        del self.data['CLINICAL_SYNDROME']
        del self.data['Indication for intervention']
        del self.data['Presenting ECG']
        del self.data['Thrombolysis']
        del self.data['COMPLICATION']
        del self.data['Replacement Device?']
        del self.data['Replacement Reason']
        del self.data['Device type']
        del self.data['Device type.1']
        del self.data['Indication for stent']
        del self.data['LMS Protected']
        del self.data['LENGTH_STAY_PROC_TO_DISCH']
        del self.data['Patient_DB_ID']
        del self.data['Procedure_Unique_Identifier']


In [32]:
#Ignore the warning message
warnings.filterwarnings('ignore')
#Read our original data
doc = pd.read_excel(filepath)
#Save the data after propressing step.
new_doc = propressing(doc).data

C:\Ipython\Anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Ipython\Anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Ipython\Anaconda2\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Ipython

In [5]:
#Tkinter: Select the method to balance the data.
msg ="Choose the methods to resample the unbalanced data"
title = "Methods"
choices = ["Under/Oversampling", "SMOTE Algorithm","Close"]
choice = buttonbox(msg, title, choices)

In [6]:
class do_balance:
    def __init__(self,method,data):
        self.method = method
        self.data = data
        if self.method == 'Under/Oversampling':
            self.databal = self.uo(self.data)
        elif self.method == 'SMOTE Algorithm':
            self.databal = self.smote(self.data)
        elif self.method == 'Close':
            self.data = self.data
            print 'The process ends. No data is changed.'
            
    def uo( self,data):
        #Under/Oversampling method. we enlarge the sample of dead and reduce the sample of alive.
        out_alive = self.data[self.data['OUTCOME'] == 1]
        out_dead = self.data[self.data['OUTCOME'] == 2]
        out_dead_new = pd.concat([out_dead]*5)
        out_alive_new = out_alive[0:1265]
        self.data = pd.concat([out_alive_new,out_dead_new])
        
    def smote( self,data ):
        #SMOTE Method
        newdata = pd.concat([self.data[self.data['OUTCOME'] ==1],self.data[self.data['OUTCOME'] ==2]])
        y_train = newdata['OUTCOME']
        predictors = self.data.columns
        for i in predictors:
            if i == 'OUTCOME':
                predictors.drop([i])
        x_train = newdata[predictors]
        sm = SMOTE(kind='regular')
        x_resampled, y_resampled = sm.fit_sample(x_train, y_train)
        x_resampled = pd.DataFrame(x_resampled)
        x_resampled.columns = predictors
        y_resampled = pd.DataFrame(y_resampled)
        y_resampled.columns = ['OUTCOME']
        self.data = pd.concat([x_resampled,y_resampled],axis = 1)

In [7]:
#save balanced data as docc
docc = do_balance(choice,doc).data

In [44]:
#Tkinter: To choose the local address to save the file
Tk().withdraw()
savepath = tkFileDialog.asksaveasfilename(defaultextension='.xlsx',initialdir = '/' ,title = "select local address to save file",filetypes = (('csv files','xlsx files'),('all files','*.*')))

In [163]:
#Save file 
writer = pd.ExcelWriter(savepath)
docc.to_excel(writer, sheet_name='Sheet1')
writer.save()